# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [131]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Generate the mathematical model

In [152]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [153]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Alkylation Process

**Parameters:**

* **Cost_Olefin:** Cost per barrel of olefin feed.
* **Cost_Isobutane:** Cost per barrel of isobutane.
* **Cost_Acid:** Cost per thousand pounds of acid.
* **Revenue_Alkylate:** Revenue per barrel of alkylate.

**Decision Variables:**

* **Olefin_Feed:** Olefin feed (barrels per day).
* **Isobutane_Recycle:** Isobutane recycle (barrels per day).
* **Acid_Addition_Rate:** Acid addition rate (thousands of pounds per day).
* **Alkylate_Yield:** Alkylate yield (barrels per day).
* **Isobutane_Makeup:** Isobutane makeup (barrels per day).
* **Acid_Strength:** Acid strength (weight percent).
* **Motor_Octane_Number:** Motor octane number.
* **Ext_Isobutane_Olefin_Ratio:** External isobutane-to-olefin ratio.
* **Acid_Dilution_Factor:** Acid dilution factor.
* **F4_Performance_Number:** F-4 performance number.

**Objective Function:**

Maximize daily profit:

```
Profit = Revenue_Alkylate * Alkylate_Yield * Motor_Octane_Number 
          - Cost_Olefin * Olefin_Feed 
          - Cost_Isobutane * (Isobutane_Recycle + Isobutane_Makeup) 
          - Cost_Acid * Acid_Addition_Rate 
```

**Constraints:**

**1. Regression Formula Deviations:**

* `0.98 * Alkylate_Yield_Calc <= Alkylate_Yield <= 1.02 * Alkylate_Yield_Calc`
* `0.99 * Motor_Octane_Number_Calc <= Motor_Octane_Number <= 1.01 * Motor_Octane_Number_Calc`
* `0.95 * Acid_Dilution_Factor_Calc <= Acid_Dilution_Factor <= 1.05 * Acid_Dilution_Factor_Calc`
* `0.9 * F4_Performance_Number_Calc <= F4_Performance_Number <= 1.1 * F4_Performance_Number_Calc`

**2. Alkylate Yield Calculation:**

* `Alkylate_Yield_Calc = Olefin_Feed * (1.12 + 0.13167 * Ext_Isobutane_Olefin_Ratio - 0.00667 * Ext_Isobutane_Olefin_Ratio^2)`

**3. Motor Octane Number Calculation:**

* `Motor_Octane_Number_Calc = 86.35 + 1.098 * Ext_Isobutane_Olefin_Ratio - 0.038 * Ext_Isobutane_Olefin_Ratio^2 + 0.325 * (Acid_Strength - 89)`

**4. Acid Dilution Factor Calculation:**

* `Acid_Dilution_Factor_Calc = 35.82 - 0.222 * F4_Performance_Number`

**5. F-4 Performance Number Calculation:**

* `F4_Performance_Number_Calc = -133 + 3 * Motor_Octane_Number`

**6. External Isobutane-to-Olefin Ratio:**

* `Ext_Isobutane_Olefin_Ratio = (Isobutane_Recycle + Isobutane_Makeup) / Olefin_Feed`

**7. Acid Strength:**

* `Acid_Strength = (93000 * Acid_Addition_Rate) / (Acid_Addition_Rate * Acid_Dilution_Factor + 1000 * Acid_Addition_Rate)`

**8. Alkylate Yield Balance:**

* `1.22 * Alkylate_Yield = Olefin_Feed + Isobutane_Makeup`

**9. Variable Bounds:**

* Define lower and upper bounds for each decision variable based on process constraints (e.g., minimum and maximum flow rates, allowable acid strength range, etc.)

This model incorporates the regression formulas, their allowed deviations, and the specific constraints of the alkylation process. By solving this optimization problem, you can determine the values for each decision variable that maximize the daily profit of the alkylation process. 


## 3. Generate the pyomo code

In [154]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [155]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data if available)
Cost_Olefin = 50  # $/barrel
Cost_Isobutane = 45  # $/barrel
Cost_Acid = 100  # $/thousand pounds
Revenue_Alkylate = 70  # $/barrel

# Model Initialization
model = pyo.ConcreteModel()

# Decision Variables
model.Olefin_Feed = pyo.Var(bounds=(1000, 5000), doc="Olefin feed (barrels per day)") # Sample bounds
model.Isobutane_Recycle = pyo.Var(bounds=(500, 2500), doc="Isobutane recycle (barrels per day)") # Sample bounds
model.Acid_Addition_Rate = pyo.Var(bounds=(10, 50), doc="Acid addition rate (thousands of pounds per day)") # Sample bounds
model.Alkylate_Yield = pyo.Var(bounds=(1000, 6000), doc="Alkylate yield (barrels per day)") # Sample bounds
model.Isobutane_Makeup = pyo.Var(bounds=(100, 500), doc="Isobutane makeup (barrels per day)") # Sample bounds
model.Acid_Strength = pyo.Var(bounds=(88, 92), doc="Acid strength (weight percent)") # Sample bounds
model.Motor_Octane_Number = pyo.Var(bounds=(90, 95), doc="Motor octane number") # Sample bounds
model.Ext_Isobutane_Olefin_Ratio = pyo.Var(bounds=(3, 10), doc="External isobutane-to-olefin ratio") # Sample bounds
model.Acid_Dilution_Factor = pyo.Var(bounds=(30, 40), doc="Acid dilution factor") # Sample bounds
model.F4_Performance_Number = pyo.Var(bounds=(130, 150), doc="F-4 performance number") # Sample bounds

# Objective Function
def profit(model):
    return Revenue_Alkylate * model.Alkylate_Yield * model.Motor_Octane_Number \
           - Cost_Olefin * model.Olefin_Feed \
           - Cost_Isobutane * (model.Isobutane_Recycle + model.Isobutane_Makeup) \
           - Cost_Acid * model.Acid_Addition_Rate
model.Profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints

# 1. Regression Formula Deviations
model.Alkylate_Yield_Deviation = pyo.Constraint(expr= 0.98 * model.Alkylate_Yield <= model.Olefin_Feed * (1.12 + 0.13167 * model.Ext_Isobutane_Olefin_Ratio - 0.00667 * model.Ext_Isobutane_Olefin_Ratio**2) <= 1.02 * model.Alkylate_Yield)
model.Motor_Octane_Number_Deviation = pyo.Constraint(expr= 0.99 * model.Motor_Octane_Number <= 86.35 + 1.098 * model.Ext_Isobutane_Olefin_Ratio - 0.038 * model.Ext_Isobutane_Olefin_Ratio**2 + 0.325 * (model.Acid_Strength - 89) <= 1.01 * model.Motor_Octane_Number)
model.Acid_Dilution_Factor_Deviation = pyo.Constraint(expr= 0.95 * model.Acid_Dilution_Factor <= 35.82 - 0.222 * model.F4_Performance_Number <= 1.05 * model.Acid_Dilution_Factor)
model.F4_Performance_Number_Deviation = pyo.Constraint(expr= 0.9 * model.F4_Performance_Number <= -133 + 3 * model.Motor_Octane_Number <= 1.1 * model.F4_Performance_Number)

# 2. Alkylate Yield Calculation (Embedded in Deviation Constraint)

# 3. Motor Octane Number Calculation (Embedded in Deviation Constraint)

# 4. Acid Dilution Factor Calculation (Embedded in Deviation Constraint)

# 5. F-4 Performance Number Calculation (Embedded in Deviation Constraint)

# 6. External Isobutane-to-Olefin Ratio
model.Ext_Isobutane_Olefin_Ratio_Calc = pyo.Constraint(expr= model.Ext_Isobutane_Olefin_Ratio == (model.Isobutane_Recycle + model.Isobutane_Makeup) / model.Olefin_Feed)

# 7. Acid Strength
model.Acid_Strength_Calc = pyo.Constraint(expr= model.Acid_Strength == (93000 * model.Acid_Addition_Rate) / (model.Acid_Addition_Rate * model.Acid_Dilution_Factor + 1000 * model.Acid_Addition_Rate))

# 8. Alkylate Yield Balance
model.Alkylate_Yield_Balance = pyo.Constraint(expr= 1.22 * model.Alkylate_Yield == model.Olefin_Feed + model.Isobutane_Makeup)

# Solve the Model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print Results
print(results)
print("Profit = ", pyo.value(model.Profit))
print("Olefin Feed = ", pyo.value(model.Olefin_Feed))
print("Isobutane Recycle = ", pyo.value(model.Isobutane_Recycle))
# ... Print other decision variable values

```

**Explanation of the Code:**

1. **Import Pyomo:** The first line imports the Pyomo library.
2. **Sample Data:** The code includes placeholder values for parameters like `Cost_Olefin`, `Cost_Isobutane`, etc. You should replace these with your actual data.
3. **Model Initialization:** A `ConcreteModel` is created to represent the optimization problem.
4. **Decision Variables:** Each decision variable is defined using `pyo.Var`, with appropriate bounds. The sample bounds provided should be replaced with your actual process limitations.
5. **Objective Function:** The `profit` function defines the objective of maximizing daily profit based on the given formula. The `pyo.Objective` component sets this function as the model's objective with the `maximize` sense.
6. **Constraints:** Each constraint is implemented using `pyo.Constraint`. The code embeds the calculations for Alkylate Yield, Motor Octane Number, Acid Dilution Factor, and F-4 Performance Number directly within the deviation constraints for efficiency.
7. **Solver:** The code selects the `ipopt` solver, which is suitable for nonlinear optimization problems. You can choose a different solver if needed.
8. **Solve and Print:** The model is solved, and the results are printed.

This Pyomo code represents the mathematical model for the alkylation process optimization. Remember to replace the sample data with your actual data and adjust the variable bounds to reflect the real-world limitations of your process. 


## 4. Input problem data and try running the generated code

In [156]:
import pyomo.environ as pyo

# Sample Data (Replace with actual data if available)
Cost_Olefin = 50  # $/barrel
Cost_Isobutane = 45  # $/barrel
Cost_Acid = 100  # $/thousand pounds
Revenue_Alkylate = 70  # $/barrel

# Model Initialization
model = pyo.ConcreteModel()

# Decision Variables
model.Olefin_Feed = pyo.Var(bounds=(1000, 5000), doc="Olefin feed (barrels per day)") # Sample bounds
model.Isobutane_Recycle = pyo.Var(bounds=(500, 2500), doc="Isobutane recycle (barrels per day)") # Sample bounds
model.Acid_Addition_Rate = pyo.Var(bounds=(10, 50), doc="Acid addition rate (thousands of pounds per day)") # Sample bounds
model.Alkylate_Yield = pyo.Var(bounds=(1000, 6000), doc="Alkylate yield (barrels per day)") # Sample bounds
model.Isobutane_Makeup = pyo.Var(bounds=(100, 500), doc="Isobutane makeup (barrels per day)") # Sample bounds
model.Acid_Strength = pyo.Var(bounds=(88, 92), doc="Acid strength (weight percent)") # Sample bounds
model.Motor_Octane_Number = pyo.Var(bounds=(90, 95), doc="Motor octane number") # Sample bounds
model.Ext_Isobutane_Olefin_Ratio = pyo.Var(bounds=(3, 10), doc="External isobutane-to-olefin ratio") # Sample bounds
model.Acid_Dilution_Factor = pyo.Var(bounds=(30, 40), doc="Acid dilution factor") # Sample bounds
model.F4_Performance_Number = pyo.Var(bounds=(130, 150), doc="F-4 performance number") # Sample bounds

# Objective Function
def profit(model):
    return Revenue_Alkylate * model.Alkylate_Yield * model.Motor_Octane_Number \
           - Cost_Olefin * model.Olefin_Feed \
           - Cost_Isobutane * (model.Isobutane_Recycle + model.Isobutane_Makeup) \
           - Cost_Acid * model.Acid_Addition_Rate
model.Profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints

# 1. Regression Formula Deviations
model.Alkylate_Yield_Deviation = pyo.Constraint(expr= 0.98 * model.Alkylate_Yield <= model.Olefin_Feed * (1.12 + 0.13167 * model.Ext_Isobutane_Olefin_Ratio - 0.00667 * model.Ext_Isobutane_Olefin_Ratio**2) <= 1.02 * model.Alkylate_Yield)
model.Motor_Octane_Number_Deviation = pyo.Constraint(expr= 0.99 * model.Motor_Octane_Number <= 86.35 + 1.098 * model.Ext_Isobutane_Olefin_Ratio - 0.038 * model.Ext_Isobutane_Olefin_Ratio**2 + 0.325 * (model.Acid_Strength - 89) <= 1.01 * model.Motor_Octane_Number)
model.Acid_Dilution_Factor_Deviation = pyo.Constraint(expr= 0.95 * model.Acid_Dilution_Factor <= 35.82 - 0.222 * model.F4_Performance_Number <= 1.05 * model.Acid_Dilution_Factor)
model.F4_Performance_Number_Deviation = pyo.Constraint(expr= 0.9 * model.F4_Performance_Number <= -133 + 3 * model.Motor_Octane_Number <= 1.1 * model.F4_Performance_Number)

# 2. Alkylate Yield Calculation (Embedded in Deviation Constraint)

# 3. Motor Octane Number Calculation (Embedded in Deviation Constraint)

# 4. Acid Dilution Factor Calculation (Embedded in Deviation Constraint)

# 5. F-4 Performance Number Calculation (Embedded in Deviation Constraint)

# 6. External Isobutane-to-Olefin Ratio
model.Ext_Isobutane_Olefin_Ratio_Calc = pyo.Constraint(expr= model.Ext_Isobutane_Olefin_Ratio == (model.Isobutane_Recycle + model.Isobutane_Makeup) / model.Olefin_Feed)

# 7. Acid Strength
model.Acid_Strength_Calc = pyo.Constraint(expr= model.Acid_Strength == (93000 * model.Acid_Addition_Rate) / (model.Acid_Addition_Rate * model.Acid_Dilution_Factor + 1000 * model.Acid_Addition_Rate))

# 8. Alkylate Yield Balance
model.Alkylate_Yield_Balance = pyo.Constraint(expr= 1.22 * model.Alkylate_Yield == model.Olefin_Feed + model.Isobutane_Makeup)

# Solve the Model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print Results
print(results)
print("Profit = ", pyo.value(model.Profit))
print("Olefin Feed = ", pyo.value(model.Olefin_Feed))
print("Isobutane Recycle = ", pyo.value(model.Isobutane_Recycle))

PyomoException: Cannot convert non-constant Pyomo expression (0.98*Alkylate_Yield  <=  Olefin_Feed*(1.12 + 0.13167*Ext_Isobutane_Olefin_Ratio - 0.00667*Ext_Isobutane_Olefin_Ratio**2)) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [157]:
print(response.text)

## Mathematical Optimization Model for Alkylation Process

**Parameters:**

* **Cost_Olefin:** Cost per barrel of olefin feed.
* **Cost_Isobutane:** Cost per barrel of isobutane.
* **Cost_Acid:** Cost per thousand pounds of acid.
* **Revenue_Alkylate:** Revenue per barrel of alkylate.

**Decision Variables:**

* **Olefin_Feed:** Olefin feed (barrels per day).
* **Isobutane_Recycle:** Isobutane recycle (barrels per day).
* **Acid_Addition_Rate:** Acid addition rate (thousands of pounds per day).
* **Alkylate_Yield:** Alkylate yield (barrels per day).
* **Isobutane_Makeup:** Isobutane makeup (barrels per day).
* **Acid_Strength:** Acid strength (weight percent).
* **Motor_Octane_Number:** Motor octane number.
* **Ext_Isobutane_Olefin_Ratio:** External isobutane-to-olefin ratio.
* **Acid_Dilution_Factor:** Acid dilution factor.
* **F4_Performance_Number:** F-4 performance number.

**Objective Function:**

Maximize daily profit:

```
Profit = Revenue_Alkylate * Alkylate_Yield * Motor_Octan

In [158]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data if available)
Cost_Olefin = 50  # $/barrel
Cost_Isobutane = 45  # $/barrel
Cost_Acid = 100  # $/thousand pounds
Revenue_Alkylate = 70  # $/barrel

# Model Initialization
model = pyo.ConcreteModel()

# Decision Variables
model.Olefin_Feed = pyo.Var(bounds=(1000, 5000), doc="Olefin feed (barrels per day)") # Sample bounds
model.Isobutane_Recycle = pyo.Var(bounds=(500, 2500), doc="Isobutane recycle (barrels per day)") # Sample bounds
model.Acid_Addition_Rate = pyo.Var(bounds=(10, 50), doc="Acid addition rate (thousands of pounds per day)") # Sample bounds
model.Alkylate_Yield = pyo.Var(bounds=(1000, 6000), doc="Alkylate yield (barrels per day)") # Sample bounds
model.Isobutane_Makeup = pyo.Var(bounds=(100, 500), doc="Isobutane makeup (barrels per day)") # Sample bounds
model.Acid_Strength = pyo.Var(bounds=(88, 92), doc="Acid strength (weight percent)") # Sample bounds
model.Motor_Octane_Number = pyo.Var(bounds=